# 必要な計算環境

* Linux OS
* Pascal, Volta, Turing, 或いは NVIDIA Ampere 世代 GPU
* Nvidia Driver
* Docker

# 事前準備

* [NGC](https://catalog.ngc.nvidia.com/?filters=&orderBy=weightPopularDESC&query=&page=&pageSize=)の登録と[APIキー](https://org.ngc.nvidia.com/setup/api-key)の取得は済みましたか？
* [Weights and Biases]((https://docs.wandb.ai/quickstart)) の登録と[APIキー](https://docs.wandb.ai/guides/track/public-api-guide#authentication)の取得は済みましたか？

### NGC のセットアップ
NGCからモデルをダウンロードしたい場合は、APIキーが必要になりますので、取得してください。
### Weights and Biases のセットアップ
モデルのトレーニング進行状況やチャートは、 [Weights and Biases](https://docs.wandb.ai/quickstart) を通じて可視化できます。ログを有効にするために、 [API キー](https://docs.wandb.ai/guides/track/public-api-guide#authentication)を設定してください。

In [ ]:
import os
#os.environ["WANDB_ENTITY"]=""
os.environ["WANDB_PROJECT"]="BioNeMo_protein_LLM_pretraining"

# GPUの確認

In [ ]:
!nvidia-smi

# データの前処理

ESM-2nv のトレーニング中には 2 つのデータセットを使用します。Uniref50 はトレーニング、検証、テスト用に分割されています。データのサイズと多様性を増加させるために、トレーニング中に Uniref50 のミニバッチがサンプリングされますが、このバッチの各シーケンスは、対応する Uniref90 クラスターのシーケンスに置き換えられます。詳細は「Language models of protein sequences at the scale of evolution enable accurate structure prediction (進化の規模でのタンパク質配列の言語モデルが正確な構造予測を実現)」を参照してください。Uniref90 はトレーニング時にのみ使用され、検証やテストには使用されません。

## 元データの取得

データの前処理を行う前に、以下のコマンドでサンプルデータを解凍してください。

In [ ]:
cd /workspace/bionemo/examples/tests/test_data/

In [ ]:
!unzip uniref202104_esm2_qc_test200_val200.zip

In [ ]:
import wandb
with wandb.init(name="data_upload") as run:
    artifact = wandb.Artifact(
        name="uniref202104_esm2_qc_test200_val200",
        type="dataset",
        description="uniref202104_esm2_qc_test200_val200 from BioNeMo examples",
        metadata={"path":"/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200.zip"},
    )
    artifact.add_dir("/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200")
    run.log_artifact(artifact)

次に、下記の場所にあるコードを実行してデータの前処理を行いますが、その前に、データのパスを設定する必要があります。

/workspace/bionemo/examples/protein/esm2nv/pretrain.py

データのパスを正しく設定するために、下記のYAML設定ファイルを変更するか、データの前処理やモデルトレーニングを行う際にコマンドの一部としてそれらのパラメータを更新することができます。

/workspace/bionemo/examples/protein/esm2nv/conf/base_config.yaml

では、以下のコマンドでデータの前処理を行います。

In [ ]:
cd /workspace/bionemo

In [ ]:
!python examples/protein/esm2nv/pretrain.py\
 --config-path=conf\
 --config-name=pretrain_esm2_650M\
 ++do_training=False\
 ++exp_manager.wandb_logger_kwargs.name='preproceed_data_upload'\
 ++exp_manager.wandb_logger_kwargs.job_type='data_upload'\
 ++wandb_artifacts.wandb_use_artifact_path='${oc.env:WANDB_ENTITY}/${oc.env:WANDB_PROJECT}/uniref202104_esm2_qc_test200_val200:v0'\
 ++wandb_artifacts.wandb_log_artifact_name='uniref202104_esm2_qc_test200_val200_preprocessed'\
 ++model.data.val_size=500\
 ++model.data.test_size=100\
 ++model.data.uf50_datapath=/uniref50_train_filt.fasta\
 ++model.data.uf90_datapath=/ur90_ur50_sampler.fasta\
 ++model.data.cluster_mapping_tsv=/mapping.tsv\
 ++model.data.dataset_path=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uf50\
 ++model.data.uf90.uniref90_path=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uf90\
 ++model.data.train.uf50_datapath=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uniref50_train_filt.fasta\
 ++model.data.train.uf90_datapath=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/ur90_ur50_sampler.fasta\
 ++model.data.train.cluster_mapping_tsv=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/mapping.tsv\
 ++model.data.val.uf50_datapath=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uniref50_train_filt.fasta\
 ++model.data.test.uf50_datapath=/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uniref50_train_filt.fasta

-- で始まるパラメータは、コマンドライン引数として pretrain.py に渡されます。例えば、 config-path と config-name は、設定 YAML ファイルのフォルダとyaml ファイル名を指定します。このパスは pretrain.py に対して相対的です。conf はexamples/protein/esm2nv/confを指し、pretrain_esm2_650M はexamples/protein/esm2nv/conf/pretrain_esm2_650M.yaml を指します。 


++ で始まるパラメータは、YAML ファイルで設定可能です。 例えば、base_config.yaml から継承された pretrain_esm2_650M.yaml では、以下のパラメータを見つけることができます： 

* do_training: データの前処理のみを行い、トレーニングは行わないように False に設定します。 
* exp_manager.wandb_logger_kwargs.name: wandbのrun nameを指定します
* exp_manager.wandb_logger_kwargs.job_type: wandbのjob_typeを指定します(job_typeは後でrunの情報を整理する時に役立つメタデータになります。これによって実装が変わることはありません。)
* wandb_artifacts.wandb_use_artifact_path: 前処理をするデータのwandbのartifactsのpathを設定します。
* wandb_artifacts.wandb_log_artifact_name: 前処理をした後のデータを保存する際のwandbのartifacts名を指定します
* model.data.val_size と model.data.test_size: 検証およびテストデータセットのサイズを指定します。 
* model.data.uf50_datapath: uniref50 fasta ファイルへのパスを指定します。
* model.data.uf90_datapath: uniref90 fasta ファイルへのパスを指定します。 
* model.data.cluster_mapping_tsv: uniref50 クラスターを uniref90 シーケンスにマッピングするファイルへのパスを指定します。 
* model.data.dataset_path: 前処理された uniref50 データの出力ディレクトリへのパスを指定します。このフォルダはトレイン、検証、テストの分割を含むことになります。 
* model.data.uf90.uniref90_path: 前処理されたuniref90 データの出力ディレクトリへのパスを指定します。このフォルダは u90_csvs というフォルダのみを持ち、uniref90 はトレーニングでのみ使用されるため、トレイン/テスト/検証の分割は持ちません。 
* model.data.train.uf50_datapath: uniref50 fasta ファイルへのパスを指定します。
* model.data.train.uf90_datapath: uniref90 fasta ファイルへのパスを指定します。 
* model.data.train.cluster_mapping_tsv: uniref50 クラスターを uniref90 シーケンスにマッピングするファイルへのパスを指定します。 
* model.data.val.uf50_datapath: uniref50 fasta ファイルへのパスを指定します。
* model.data.test.uf50_datapath: uniref50 fasta ファイルへのパスを指定します。

また、上記のようにコマンドラインを通じて引数を上書きするのではなく、直接 YAML ファイルを変更することもできます。処理が完了すると、前処理されたデータは /workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uf50/uf50/ にあります。

もし独自のデータを利用して事前学習、ファインチューニングや推論をしたい場合は、パスを/workspace/bionemo/mydata/ に指定してください。ただし、データの構造やフォーマットをサンプルデータに揃える必要があります。

# モデルの事前学習

今回の例で UniRef50 と UniRef90 のサブセットの前処理が完了したので、ESM-2nv モデルの事前学習を開始することができます。BioNeMo Framework では、事前学習されたモデルのチェックポイントが提供されています。例えば、ESM-1nv、ESM-2nv（650m、3b、15b）、ProtT5nv、MegaMolBART などです。これらのモデルの重みは NVIDIA の NGC からダウンロードできます。すでにモデルのチェックポイントを持っている場合は、事前学習を再開することも可能です。

では、以下のコマンドでモデルを事前学習します。

In [ ]:
cd /workspace/bionemo

In [10]:
import torch
torch.cuda.empty_cache()

In [ ]:
!python examples/protein/esm2nv/pretrain.py \
 --config-path=conf \
 --config-name=pretrain_esm2_650M \
 ++do_training=True \
 ++do_testing=True \
 ++wandb_artifacts.wandb_use_artifact_path='${oc.env:WANDB_ENTITY}/${oc.env:WANDB_PROJECT}/uniref202104_esm2_qc_test200_val200_preprocessed:v0'\
 ++model.data.dataset_path=/ \
 ++model.data.uf90.uniref90_path=/uf90 \
 ++trainer.devices=1 \
 ++model.tensor_model_parallel_size=1 \
 ++model.micro_batch_size=8 \
 ++trainer.max_steps=10 \
 ++trainer.val_check_interval=1 \
 ++exp_manager.create_wandb_logger=True \
 ++exp_manager.checkpoint_callback_params.save_top_k=10

パラメータの説明：
* do_training: モデルをトレーニングするために True に設定します。これはデータが前処理されたことを前提としています。
* do_testing: テストをスキップするために False に設定します。
* wandb_artifacts.wandb_use_artifact_path: 学習に使用するデータセットのパスを入力します
* model.data.dataset_path: トレーニング/検証/テスト分割が含まれる前処理済みのuniref50データフォルダのパスを指定します。
* model.data.uf90.uniref90_path: 前処理済みの uniref90 データのパスを指定します。このフォルダには u90_csvs という別のフォルダが含まれている必要があります。u90_csvs フォルダ内には、x000.csv から x049.csv のファイルが存在する必要があります。
* trainer.devices: 使用するGPUの数を指定します。
* model.tensor_model_parallel_size: テンソルモデル並列サイズを設定します。
* model.micro_batch_size: バッチサイズを設定します。メモリエラーが発生しない限り、これをできるだけ増やします。
* trainer.max_steps: トレーニングの最大ステップ数を指定します。デモのために 100 に設定しました。1 ステップ = 1 バッチの処理です。まず、total_batches = サンプル総数 / バッチサイズを計算します。N エポックでトレーニングしたい場合は、max_steps をN * total_batches に設定します。
* trainer.val_check_interval: 検証セットを実行する間隔を指定します。
* exp_manager.create_wandb_logger: wandb へのログ記録を無効にするために False に設定します。True の場合は、wandb API キーを提供する必要があります。
* exp_manager.checkpoint_callback_params.save_top_k: 保存する最適のチェックポイントの数を指定します。
トレーニングされた結果が /workspace/bionemo/results/nemo_experiments/ に保存されます。


# ファインチューニング

In [1]:
cd /workspace/bionemo

/workspace/bionemo


In [2]:
#一度Kernelをrestartしてください
import os
os.environ["BIONEMO_HOME"]="/workspace/bionemo"
#os.environ["WANDB_ENTITY"]=""
os.environ["WANDB_PROJECT"]="BioNeMo_protein_LLM_finetuning"

BioNemo Framework のサンプルコードは三つの下流タスクを提供しています。一つ目はタンパク質の 10 の細胞内局在部位を予測すること、二つ目はタンパク質の融解温度を予測すること、三つ目はタンパク質の 3 状態構造を予測することです。本記事では三つ目のタスクを例として説明します。具体的には、シーケンスの各アミノ酸について、それがヘリックス、シート、またはコイルのどれにあるかを予測します。
サンプルデータは下記のフォルダにあります。

/workspace/bionemo/examples/tests/test_data/protein/downstream/

In [3]:
!wget -q -O /tmp/ngccli_linux.zip --content-disposition https://api.ngc.nvidia.com/v2/resources/nvidia/ngc-apps/ngc_cli/versions/3.38.0/files/ngccli_linux.zip && unzip -o /tmp/ngccli_linux.zip -d /tmp && chmod u+x /tmp/ngc-cli/ngc && rm /tmp/ngccli_linux.zip

Archive:  /tmp/ngccli_linux.zip
   creating: /tmp/ngc-cli/
   creating: /tmp/ngc-cli/boto3/
   creating: /tmp/ngc-cli/boto3/examples/
  inflating: /tmp/ngc-cli/boto3/examples/s3.rst  
  inflating: /tmp/ngc-cli/boto3/examples/cloudfront.rst  
   creating: /tmp/ngc-cli/boto3/data/
   creating: /tmp/ngc-cli/boto3/data/dynamodb/
   creating: /tmp/ngc-cli/boto3/data/dynamodb/2012-08-10/
  inflating: /tmp/ngc-cli/boto3/data/dynamodb/2012-08-10/resources-1.json  
   creating: /tmp/ngc-cli/boto3/data/sqs/
   creating: /tmp/ngc-cli/boto3/data/sqs/2012-11-05/
  inflating: /tmp/ngc-cli/boto3/data/sqs/2012-11-05/resources-1.json  
   creating: /tmp/ngc-cli/boto3/data/opsworks/
   creating: /tmp/ngc-cli/boto3/data/opsworks/2013-02-18/
  inflating: /tmp/ngc-cli/boto3/data/opsworks/2013-02-18/resources-1.json  
   creating: /tmp/ngc-cli/boto3/data/iam/
   creating: /tmp/ngc-cli/boto3/data/iam/2010-05-08/
  inflating: /tmp/ngc-cli/boto3/data/iam/2010-05-08/resources-1.json  
   creating: /tmp/ngc-cli/

次に、事前学習済みモデルをダウンロードします。モデルをダウンロードするため、ngc をインストールして、ngc config を設定する必要があります。

/tmp/ngc-cli/ngc config set

<順番にAPI キー、CLI output format, org, team, aceを入力してください。API キーはNGCのAPIキーを入力してください。Org は ’no-org’ 以外のものを選択して、その他は「Enter」でデフォルト値を入力すれば大丈夫です>

最後に、下記のコマンドを入力すれば、モデルをダウンロードできます。

In [ ]:
!python download_models.py --download_dir /workspace/bionemo/models esm2nv_650m

In [ ]:
# モデルの保存
import wandb
with wandb.init(name="model_upload") as run:
    artifact = wandb.Artifact(
        name="esm2nv_650m",
        type="model",
        description="original esm2nv_650m",
        metadata={"path":"bionemo/models/esm2nv_650M_converted.nemo"},
    )
    artifact.add_file("/workspace/bionemo/models/esm2nv_650M_converted.nemo")
    run.log_artifact(artifact)

In [ ]:
# データセットの保存
with wandb.init(name="data_upload") as run:
    artifact = wandb.Artifact(
        name="downstream_taskdataset",
        type="dataset",
        description="bionemo/examples/tests/test_data/protein/downstream",
        metadata={"path":"/workspace/bionemo/examples/tests/test_data/protein/downstream"},
    )
    artifact.add_dir("/workspace/bionemo/examples/tests/test_data/protein/downstream")
    run.log_artifact(artifact)

ダウンロードされたモデルは .nemo というファイルで、 /workspace/bionemo/models に保存されます。
そして、以下の YAML ファイルを開きます。

/workspace/bionemo/examples/protein/esm2nv/conf/downstream_flip_sec_LORA.yaml



以下のパラメータは必要に応じて設定してください。

* restore_from_path: 事前学習済みのモデルのチェックポイントの .nemo ファイルのパスに設定します。 
* trainer.devices, trainer.num_nodes: 使用する GPU とノードの数に設定します。 
* trainer.max_epochs: トレーニングしたいエポック数に設定します。 
* trainer.val_check_interval: 検証を実行するステップ数に設定します。 
* model.micro_batch_size: トレーニングのマイクロバッチサイズに設定します。 
* data.task_name: 任意の名前を設定します。 
* data.task_type: 現在のオプションには、token-level-classification、classification (sequence level)、および regression (sequence level) があります。 preprocessed_data_path: dataset_path の親フォルダのパスに設定します。
* wandb_artifacts.wandb_use_artifact_data_path: 使用するデータのwandb artifactsのpathです。
* wandb_artifacts.wandb_use_artifact_model_path: 使用するモデルのwandb artifactsのpathです。
* dataset_path: train/val/testフォルダが含まれるフォルダのパスに設定します。例えば、上記のpath/to/data というパスに設定します。
* dataset.train, dataset.val, dataset.test: CSV 名または範囲に設定します。 
* sequence_column: シーケンスを含む列の名前に設定します。例：sequence 
* target_column: ターゲットを含む列の名前に設定します。例：scl_label 
* target_size: 分類のための各ラベルのクラス数。 
* num_classes: target_size に設定します。

そして、下記のコマンドでファインチューニングを実行します。BioNemo Framework v1.4 は新たな機能として、LoRa というファインチューニングの手法追加しました。Lora は事前学習済み大規模言語モデルのすべての重みをファインチューニングするのではなく、大規模な重み行列を近似する二つの小規模な行列をファンチューニングする効率的なファインチューニングの手法です。

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
import os
import shutil

# ディレクトリが存在し、checkpointの中身がある場合は削除（以下のコードを実行するための簡易的な処置です）
checkpoint_dir = "/workspace/bionemo/results/nemo_experiments/esm2nv_flip/esm2nv_flip_secondary_structure_finetuning_encoder_frozen_False/checkpoints"
if os.path.exists(checkpoint_dir) and os.listdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
    os.makedirs(checkpoint_dir)

In [12]:
!python examples/protein/downstream/downstream_flip.py\
 --config-path="../esm2nv/conf"\
 --config-name=downstream_sec_str_LORA\
 ++trainer.devices=1\
 ++trainer.num_nodes=1\
 ++trainer.max_epochs=100\
 ++wandb_artifacts.wandb_use_artifact_data_path='${oc.env:WANDB_ENTITY}/${oc.env:WANDB_PROJECT}/downstream_taskdataset:v0'\
 ++wandb_artifacts.wandb_use_artifact_model_path='${oc.env:WANDB_ENTITY}/${oc.env:WANDB_PROJECT}/esm2nv_650m:v0'\
 ++model.data.dataset_path=/\
 ++model.restore_encoder_path=/esm2nv_650M_converted.nemo

[NeMo I 2024-06-04 01:12:18 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-06-04 01:12:18 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform
[NeMo W 2024-06-04 01:12:19 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'downstream_sec_str_LORA': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
      warnings.warn(msg, UserWarning)
    
[NeMo W 2024-06-04 01:12:19 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more

# 推論

下記の YAML ファイルを開いて、

/workspace/bionemo/examples/protein/esm2nv/conf/infer.yaml

以下の情報を更新してください。提供する事前学習済みモデルを使用するか、自分でトレーニングしたモデルを使用することができます。

In [ ]:
downstream_task:
 restore_from_path: "${oc.env:BIONEMO_HOME}/models/protein/esm2nv/esm2nv_650M_converted.nemo" # 事前学習済みモデルのパス

下記の Jupyter Notebook を開いて、

/workspace/bionemo/examples/protein/esm2nv/nbs/inference_interactive.ipynb


セルを順次実行してください。


# 独自データの準備（オプション）

ローカルマシンのディレクトリ /home/bionemo/ に、train、val、testというフォルダを準備してください。それぞれのフォルダの中に、x000.csv, x001.csv,… という名前のファイルを準備してください。
## 事前学習データの準備
事前学習データのフォーマットについては、下記のファイルを参考にしてください。

/workspace/bionemo/examples/tests/test_data/uniref202104_esm2_qc_test200_val200/uf50/train/x000.csv

recode_id と sequence 列が必須で、他の列は任意です。
また、下記の YAML ファイルの data の部分についてもデータの数やパスなどを変更する必要があります。

/workspace/bionemo/examples/protein/esm2nv/conf/base_config.yaml

## ファインチューニングデータの準備
ファインチューニングデータのフォーマットについては、下記のファイルを参考にしてください。

/workspace/bionemo/examples/tests/test_data/protein/downstream/train/x000.csv

id、sequence と target(3state か 8state) 列が必須で、他の列は任意です。
また、下記の YAML ファイルの data の部分についてもデータの数、パス、ターゲット情報などを変更する必要があります。

/workspace/bionemo/examples/protein/esm2nv/conf/downstream_sec_str_LORA.yaml
